In [4]:
import pandas as pd
pd.set_option("display.max_columns", 25)

# Load the Excel file
file_path = './Datasets/Personality Survey Data.xlsx'

# Load the 'Personal Characteristics Survey' sheet into a pandas DataFrame
survey_data = pd.read_excel(file_path, sheet_name='Personal Characteristics Survey')

# Remove the first few rows to clean up the headers
cleaned_survey_data = pd.read_excel(file_path, sheet_name='Personal Characteristics Survey', header=1)


# Drop any rows where all columns are NaN
cleaned_survey_data.dropna(how='all', inplace=True)

# Reset index after dropping NaN rows
cleaned_survey_data.reset_index(drop=True, inplace=True)

# Remove columns where all values are NaN
cleaned_survey_data = cleaned_survey_data.dropna(axis=1, how='all')



# Reassign the first row to be the column headers
cleaned_survey_data.columns = cleaned_survey_data.iloc[0]

# Drop the first row now that it's set as column headers
cleaned_survey_data = cleaned_survey_data.drop(0).reset_index(drop=True)


retailer_index = cleaned_survey_data[cleaned_survey_data['PLAYER NAME'] == 'retailer22'].index.min()

# If the index is found, delete all rows after the first instance of "retailer22"
if pd.notna(retailer_index):
    cleaned_survey_data = cleaned_survey_data.iloc[:retailer_index + 1]

columns_to_drop = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10']

# Drop those columns if they exist in the dataframe
cleaned_survey_data = cleaned_survey_data.drop(columns=columns_to_drop, errors='ignore')

cleaned_survey_data = cleaned_survey_data.rename(columns={cleaned_survey_data.columns[3]: 'Self Esteem Average'})

value_mapping = {
    'Completely Disagree': 1,
    'Somewhat Disagree': 2,
    'Moderately Disagree': 3,
    'Not Sure': 4,
    'Somewhat Agree': 5,
    'Moderately Agree': 6,
    'Completely Agree': 7
}

# Replace the values in columns R1 to R5 using the mapping
columns_to_replace = ['R1', 'R2', 'R3', 'R4', 'R5']
cleaned_survey_data[columns_to_replace] = cleaned_survey_data[columns_to_replace].replace(value_mapping)


cleaned_survey_data["Regret Scale Average"] = (cleaned_survey_data['R1'].astype(int) + cleaned_survey_data['R2'].astype(int) + cleaned_survey_data['R3'].astype(int) + cleaned_survey_data['R4'].astype(int) + cleaned_survey_data['R5'].astype(int)) / 5


cleaned_survey_data = cleaned_survey_data.dropna(axis=1, how='all')

cleaned_survey_data = cleaned_survey_data.drop(columns=['R1', 'R2', 'R3', 'R4', 'R5'], errors='ignore')

# Move "Regret Scale Average" next to "Self Esteem Average"
# First, remove the column "Regret Scale Average" and store it temporarily
regret_scale_average = cleaned_survey_data.pop('Regret Scale Average')

# Insert the "Regret Scale Average" column right after "Self Esteem Average"
self_esteem_index = cleaned_survey_data.columns.get_loc('Self Esteem Average')
cleaned_survey_data.insert(self_esteem_index + 1, 'Regret Scale Average', regret_scale_average)

rl_value_mapping = {
    'RL1': 2,
    'RL2': 1.5,
    'RL3': 1,
    'RL4': 0.5,
    'RL5': 0,
    'RL6': -0.5,
    'RL7': -1
}

# Initialize a new column "Tossing Coin Risk Averse Coefficient"
cleaned_survey_data['Tossing Coin Risk Averse Coefficient'] = None

# Assign values based on the first occurrence of RL value equal to 0

for index, row in cleaned_survey_data.iterrows():
    for rl_column in ['RL 1', 'RL 2', 'RL 3', 'RL 4', 'RL 5', 'RL 6', 'RL 7']:
        if row[rl_column] == 0:  # Check if the RL value is 0
            cleaned_survey_data.loc[index, 'Tossing Coin Risk Averse Coefficient'] = rl_value_mapping.get(rl_column.replace(' ', ''))
            break

def calculate_risk_value(row):
    # Calculate Risk1 value
    risk1_value = 1 if row['Risk1'] < 100 else 0 if row['Risk1'] == 100 else -1

    # Calculate Risk 2 value
    risk2_value = 0.5 if row['Risk 2'] == '100 TL' else -0.5 if row['Risk 2'] == 'Ticket' else 0

    # Calculate Risk 3 value
    risk3_value = 1 if row['Risk 3'] < 100 else 0 if row['Risk 3'] == 100 else -1

    # Sum Risk values
    total_risk = risk1_value + risk2_value + risk3_value
    return total_risk

# Apply the risk calculation
cleaned_survey_data['Risk Sum'] = cleaned_survey_data.apply(calculate_risk_value, axis=1)

# Calculate the "Risk Averse Coefficient" by summing the Risk Sum and Tossing Coin Risk Averse Coefficient
cleaned_survey_data['Risk Averse Coefficient'] = cleaned_survey_data['Risk Sum'] + pd.to_numeric(cleaned_survey_data['Tossing Coin Risk Averse Coefficient'], errors='coerce')

# Drop the unnecessary columns: Risk1, Risk 2, Risk 3, RL1 through RL7, and Tossing Coin Risk Averse Coefficient
columns_to_drop = ['Risk1', 'Risk 2', 'Risk 3', 'RL 1', 'RL 2', 'RL 3', 'RL 4', 'RL 5', 'RL 6', 'RL 7', 'Tossing Coin Risk Averse Coefficient']
cleaned_survey_data = cleaned_survey_data.drop(columns=columns_to_drop, errors='ignore')

risk_averse_coefficient = cleaned_survey_data.pop('Risk Averse Coefficient')

# Insert the "Risk Averse Coefficient" column right after "Regret Scale Average"
regret_scale_index = cleaned_survey_data.columns.get_loc('Regret Scale Average')
cleaned_survey_data.insert(regret_scale_index + 1, 'Risk Averse Coefficient', risk_averse_coefficient)

points = {
    'F3': -2,  # 10 TL offer acceptance
    'F4': -1,  # 20 TL offer acceptance
    'F5': 0,  # 30 TL offer acceptance
    'F6': 1,  # 40 TL offer acceptance
    'F7': 2  # 50 TL offer acceptance
}

# Calculate Fairness for Q1 and Q2 based on F6 and F7
cleaned_survey_data['Fairness_Q1'] = (100 - cleaned_survey_data['F1']) / 100
cleaned_survey_data['Fairness_Q2'] = (100 - cleaned_survey_data['F2']) / 100

# Calculate Fairness_Q3 based on weighted points from F1 to F5
cleaned_survey_data['Fairness_Q3'] = cleaned_survey_data[[col for col in points.keys()]].multiply(points.values(), axis=1).sum(axis=1) / 10

# Calculate the overall Fairness Index
cleaned_survey_data['Fairness Index'] = (cleaned_survey_data['Fairness_Q1'] + cleaned_survey_data['Fairness_Q2'] + cleaned_survey_data['Fairness_Q3']) / 3

# Drop the F1 to F7 columns
cleaned_survey_data = cleaned_survey_data.drop(columns=['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7'], errors='ignore')

cleaned_survey_data = cleaned_survey_data.drop(["Fairness_Q1", "Fairness_Q2", "Fairness_Q3"], axis=1)

cleaned_survey_data = cleaned_survey_data.loc[:, cleaned_survey_data.columns.notna()]

# Display the cleaned data
cleaned_survey_data.head()


cleaned_survey_data.to_csv('adjusted_reponse_survey.csv', index=False)

C:\Users\telat\AppData\Local\Temp\ipykernel_31436\1764912196.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_survey_data[columns_to_replace] = cleaned_survey_data[columns_to_replace].replace(value_mapping)


In [3]:
cleaned_survey_data.head(20)

,PLAYER NAME,Gender,Date of birth,Self Esteem Average,Regret Scale Average,Risk Averse Coefficient,Risk Sum,Fairness Index
0,manufacturer1,Male,1993-10-11,1.9,3.6,-0.5,-0.5,0.566667
1,manufacturer2,Female,1990-06-03,2.9,4.8,1.5,1.5,0.5
2,manufacturer3,Male,1991-07-05,1.7,5.8,3.5,2.5,0.566667
3,manufacturer4,Female,1991-08-08,2.0,4.2,1.0,0.5,0.533333
4,manufacturer5,Male,1988-05-18,2.6,5.4,3.0,2.5,0.336667
5,manufacturer6,Male,1990-09-09,1.2,5.4,0.5,-0.5,0.433333
6,manufacturer7,Female,1992-05-10,1.9,4.0,2.0,0.5,0.5
7,manufacturer8,Male,1992-04-10,2.6,5.2,-1.5,-1.5,0.466667
8,manufacturer9,Male,1990-09-11,2.8,3.4,2.5,1.5,0.55
9,manufacturer10,Female,1990-06-07,3.0,3.4,2.5,1.5,0.5


In [58]:
cleaned_survey_data["F1"].value_counts()


F1
50     20
100    11
60      3
90      3
75      2
80      2
99      1
55      1
20      1
Name: count, dtype: int64